In [12]:
import requests
import time
from bs4 import BeautifulSoup
import csv
from collections import defaultdict
import json
import sqlite3
from sqlite3 import Error
from queue import Queue 

In [13]:
# terms_dict = defaultdict(list)
terms_list = []
appDictionary = {}
relatedSearchTerms = {}

In [14]:
def commaSeparated(term):
    result = "";
    if len(term.split(' ')) > 1:
        count = len(term.split(' '));
        i = 0;
        for word in term.split(' '):
            if i > 0:
                result = result + "+";
            result = result + word;
            i = i + 1;
        return result;
    else:
        return term;

In [15]:
#Class to store app details in object format
class AppDetails:
    def __init__(self, title, description, stars, appID, imageSource, developerName):
        self.title = title
        self.description = description
        self.stars = stars
        self.appID = appID
        self.imageSource = imageSource
        self.developerName = developerName
        
    def toString(application):
        return application.title + ", " + application.appID + ", " + application.description + ", " + application.stars + ", " + application.imageSource + ", " + application.developerName

In [16]:
#DB DRIVERS

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return conn
            
            
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
sql_create_appDetailsMainTable_table = """ CREATE TABLE IF NOT EXISTS appDetailsMainTable (
                                        key text NOT NULL,
                                        appIDs text NOT NULL,
                                        relatedSearchTerms text,
                                        source text NOT NULL
                                    ); """

sql_create_appDetails_table = """ CREATE TABLE IF NOT EXISTS appDetails (
                                        appID text NOT NULL,
                                        title text NOT NULL,
                                        description text,
                                        stars text,
                                        imageSource text,
                                        developerName text,
                                        source text
                                    ); """

sql_create_relatedSearchTerms_table = """ CREATE TABLE IF NOT EXISTS relatedSearchTerms (
                                        keyTerm text PRIMARY KEY,
                                        terms text NOT NULL,
                                        source text
                                    ); """

def databaseStartUp():
    # Create connection
    conn = create_connection(r"apkdl.db")

    # Create table
    if conn is not None:
        create_table(conn, sql_create_appDetailsMainTable_table)
        create_table(conn, sql_create_appDetails_table)
        create_table(conn, sql_create_relatedSearchTerms_table)
        print("Database created!")
    else:
        print("Error! cannot create the database connection.")
    
    return conn;
        
        
def insertIntoAppDetailsMainTable(conn, task):
 
    sql = ''' INSERT INTO appDetailsMainTable(key,appIDs,relatedSearchTerms,source)
              VALUES(?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()
    return cur.lastrowid

def insertIntoAppDetails(conn, task):
 
    sql = ''' INSERT INTO appDetails(appID, title, description, stars, imageSource, developerName, source)
              VALUES(?,?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()
    return cur.lastrowid
     
conn = databaseStartUp()

In [17]:
with open('android_terms.csv','rt')as f:
  data = csv.reader(f)
  lineNumber = 0;
  for row in data:
        if lineNumber >= 1:
            key = row[2];
            terms = row[3];
            i = 0;
            for term in terms.split('"'):
                if i%2 == 0:
                    i = i+1;
                    continue
                else:
                    i = i+1;
                    terms_list.append(term)
        else:
            lineNumber = lineNumber + 1;

In [18]:
finalTermsList = []

In [19]:
finalTermsList.clear()
for term in terms_list:
    result = commaSeparated(term)
    finalTermsList.append(result)

In [20]:
len(finalTermsList)

432331

In [ ]:
# WRITTEN FOR TESTING PURPOSE PLEASE IGNORE 

payload = {'q': 'mspy', 't': 'app'}
r = requests.get('https://apk.support/search', params=payload);
print(r.url)
print(r.status_code)

In [ ]:
# TEST
soup = BeautifulSoup(r.text, 'html.parser')
app_table = soup.find_all("h3")
appNamesList = []
for app in app_table:
    appName = app.get_text()
    appNamesList.append(appName)
appDictionary["mspy"] = appNamesList


# TEST Cleanup List
for term in finalTermsList:
    if(term == ""):
        finalTermsList.remove(term)
singleWordSet = set();
for term in finalTermsList:
    if(term.find("+") == -1 and term != ""):
        singleWordSet.add(term)

In [21]:
q = Queue();
wordSet = set();
for term in finalTermsList:
    if(term != ""):
        wordSet.add(term)
print(len(wordSet))
for word in wordSet:
    q.put(word)
print(q.qsize())

In [ ]:
# FOR TEST PURPOSE
singleWordSet = {'cheater', 'cheaters', 'cheating', 'track husband', 'keylogger', 'keyloggers'}

In [ ]:
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'q': word, 't': 'app'};
    r = requests.get('https://apk.support/search', params=payload);
    soup = BeautifulSoup(r.text, 'html.parser');
    # Get App Names
    names_table = soup.find_all("div", attrs={"class": "it_column"})
    if(len(names_table) == 0):
        print("Skipping " + word)
        continue
    finalList = []
    appIDList = ""
    first = 0
    for name in names_table:
        # Developer Information
        developerPart = name.find_all("div", attrs={"class": "ss_tg"})
        developerPart = developerPart[0].find_all("a")
        developerTag = developerPart[0]['href']
        developerTag = developerTag[10:]
        developerName = developerPart[0].get_text()
        information = name.find_all("a")
        # Title
        titleTag = information[0].find_all("h3")
        title = titleTag[0].get_text()
#         print("Title is " + title)
        # Description
        descriptionTag = information[0].find_all("p")
        description = descriptionTag[0].get_text()
#         print("Description is " + description)
        # Stars
        starsTag = information[0].find_all("div", attrs = {"class" : "stars"})
        starsSpan = starsTag[0].find_all("span")
        stars = starsSpan[0]['title']
        starCount = stars[stars.rindex(' ')+1:]
#         print("Stars is " + starCount)
        # AppID
        appID = information[0]['href']
        appID = appID[4 : ]
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1
#         print("AppID:- " + appID)
        # Image Source Link
        imageTag = information[0].find_all("div", attrs={"class" : "seo_img"})
        imageTag = imageTag[0].find_all("img")
        imageSource = imageTag[0]['data-original']
#         print(" ImageSrc:- " + imageSource)
        perAppObject = AppDetails(title, description, starCount, appID, imageSource, developerName)
        
        # Insert Into App Table
        taskAppTable = (appID, title, description, stars, imageSource, developerName, 'apk.support');
        insertIntoAppDetails(conn, taskAppTable)
        
        finalList.append(perAppObject)
        
    # Suggestion Addition
    suggestionList = soup.find_all("div", attrs={"class": "suggest"})
    suggestionList = suggestionList[0].find_all("li")
    suggestions = []
    suggestionsString = "";
    i = 0
    for suggestion in suggestionList:
        suggestionName = suggestion.get_text()
        if (i != 0):
            suggestionsString = suggestionsString + ","
        suggestionsString = suggestionsString + suggestionName
        i = 1
        suggestions.append(suggestionName)
        modifiedSuggestionName = commaSeparated(suggestionName)
        if(modifiedSuggestionName not in wordSet):
#             print("Added " + modifiedSuggestionName)
            wordSet.add(modifiedSuggestionName)
            q.put(modifiedSuggestionName)
#         else:
#             print("Not Adding " + modifiedSuggestionName)
    
    #Insert Into Main Table
    taskMainTable = (word, appIDList, suggestionsString, 'apk.support');
    insertIntoAppDetailsMainTable(conn, taskMainTable)
    
    
#     appDictionary[word] = finalList
#     relatedSearchTerms[word] = suggestions
    numberOfTerms = numberOfTerms + 1
    if(numberOfTerms == 5000):
        break
    print("Completed " + word)

In [ ]:
#https://apkgk.com/
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'keyword': word}
    r = requests.get('https://apkgk.com/search/', params=payload);
    soup = BeautifulSoup(r.text, 'html.parser');
    appId = soup.find_all("ul", attrs={"class":'topic-wrap'})
    if(len(appId) == 0):
        print("Skipping " + word)
        continue
    appId = appId[0].find_all("a")
    names_table = soup.find_all('div', attrs={"class":'topic-bg'})
    if(len(names_table) == 0):
        print("Skipping " + word)
        continue
    finalList = []
    appIDList = ""
    first = 0
    counter = 0
    for name in names_table:
        appName = name.find_all("div", attrs={"class": "topic-tip-name"})
        appDesc = name.find_all("div", attrs={"class": "topic-tip-description"})
        appSrcMain  = name.find_all("div", attrs={"class": "c-lz-load"})
        imageTag = appSrcMain[0].find_all("img")
        appLink = imageTag[0]['data-src']
        #for img in appSrcMain:
        print(appId[counter]['href'])
        print(appLink)
        print(appName[0].get_text())
        print(appDesc[0].get_text())
        
        appID = appId[counter]['href']
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1
        imageSource = appLink
        description = appDesc[0].get_text()
        title = appName[0].get_text()
        starCount = "NULL"
        developerName = "NULL"
        perAppObject = AppDetails(title, description, starCount, appID, imageSource, developerName)
        
        # Insert Into App Table
        taskAppTable = (appID, title, description, starCount, imageSource, developerName, 'apkgk.com');
        insertIntoAppDetails(conn, taskAppTable)
        
        finalList.append(perAppObject)
        counter=counter+1
    #Insert Into Main Table
    suggestionsString = "NULL"
    taskMainTable = (word, appIDList, suggestionsString, 'apkgk.com');
    insertIntoAppDetailsMainTable(conn, taskMainTable)
        
        


In [ ]:
#https://www.malavida.com
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    word = word.replace('+','-')
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    #payload = {'/':word}
    r = requests.get('https://www.malavida.com/en/s/'+word);
    soup = BeautifulSoup(r.text, 'html.parser');
    appDetails = soup.find_all("section", attrs={"class":'app-list'})
    appList = soup.find_all("section", attrs={"class":'app-download'})
    counter = 0
    finalList = []
    appIDList = ""
    first = 0
    notFound = soup.find_all("section", attrs={"class":'not-found'})
    if(len(notFound)>0):
        print("Skipping " + word)
        continue
    print(len(appList))
    for app in appList:
        appSrc = app.find_all("div", attrs={"class":"title"})
        appDesc = app.find_all("p")
        imageSource = app.find_all('img')
        imageSource = imageSource[0]['src']
        appLink = appSrc[0].find_all("a")
        appID = appLink[0]['href']
        title = appLink[0].get_text()
        description = appDesc[0].get_text()
        counter = counter+1
        
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1
        starCount = "NULL"
        developerName = "NULL"
        print(title +", "+description+", "+imageSource+", "+appID)
        perAppObject = AppDetails(title, description, starCount, appID, imageSource, developerName)
        
        # Insert Into App Table
        taskAppTable = (appID, title, description, starCount, imageSource, developerName, 'malavida.com');
        insertIntoAppDetails(conn, taskAppTable)
        
        finalList.append(perAppObject)
        counter=counter+1
    #Insert Into Main Table
    suggestionsString = "NULL"
    taskMainTable = (word, appIDList, suggestionsString, 'malavida.com');
    insertIntoAppDetailsMainTable(conn, taskMainTable)
        
    
    

In [ ]:
# Map check Code
for app in appDictionary:
    appList = appDictionary[app]
    print("Entries for " + app)
    for application in appList:
        print(application.toString())
        print("------------------------------->>>")

In [ ]:
# Saving dict to file
with open('apksupport.json', 'w') as f:
    json.dump(appDictionary, f)

with open('relatedSearchTerms_apksupport.json', 'w') as f:
    json.dump(relatedSearchTerms, f)

In [ ]:
SQLite3
dataset package python

upper table -> key and set of app ids
lower table -> app id is primary key and we write to this.

In [ ]:
conn.close()

In [ ]:
# FOR APKCOMBO.COM (ERROR: CLOUDFLARE BLOCKING)
qTrial = Queue()
qTrial.put("cheating+wife")
numberOfTerms = 0
while(qTrial.empty() != True):
    word = qTrial.get()
    time.sleep(5)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'q': word};
    r = requests.get('https://apkcombo.com/search', params=payload);
    soup = BeautifulSoup(r.text, 'html.parser');
    print(soup.prettify())
    # Get App Names
    names_table = soup.find_all("div", attrs={"class": "grid-apps mb-10"})
    

In [ ]:
# FOR APKPURE.COM
qTrial = Queue()
qTrial.put("Starting hidden+tracker+on+my+husband+cell+phone")
qTrial.put("phone+controller+app")
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    checkMore = 1
    firstCheck = 1
    interval = 0
    appIDList = ""
    first = 0
    while(checkMore):
        if(firstCheck == 0):
            interval = interval + 15
        payload = {'q': word,  't': 'app', 'begin': interval};
        payload_str = "&".join("%s=%s" % (k,v) for k,v in payload.items())
        r = requests.get('https://apkpure.com/search-page', params=payload_str);
        soup = BeautifulSoup(r.text, 'html.parser')
        #print(soup.prettify())
        names_table = soup.find_all("dl", attrs={"class": "search-dl"})
        #print(names_table)
        if(len(names_table) == 0):
            print("Skipping " + word)
            break
        for name in names_table:
            dtPart = name.find_all("dt");
            ddPart = name.find_all("dd");
            aPart = dtPart[0].find_all("a");
            appID = aPart[0]['href']
            imagePart = aPart[0].find_all("img");
            imageLink = imagePart[0]['src']
            titlePart = ddPart[0].find_all("p", attrs={"class": "search-title"})
            title = titlePart[0].find("a").get_text();
            starsPart = ddPart[0].find_all("span", attrs={"class": "score"})
            stars = starsPart[0]['title']
            stars = stars.rsplit(' ', 1)[1]
            pParts = ddPart[0].find_all("p")
            developerPart = pParts[1].find_all("a")
            developerName = developerPart[0].get_text()
            if first != 0:
                appIDList = appIDList + ","
            appIDList = appIDList + appID
            first = 1
            
            appTableEntry = (appID, title, 'null', stars, imageLink, developerName, 'apkpure.com');
            insertIntoAppDetails(conn, appTableEntry)
        print("Done")
        firstCheck = 0
        if(len(names_table) == 0):
            checkMore = 0
    appMainTableEntry = (word, appIDList, 'null', 'apkpure.com');
#     print(appMainTableEntry)
    insertIntoAppDetailsMainTable(conn, appMainTableEntry)
    numberOfTerms = numberOfTerms + 1
    if(numberOfTerms == 5000):
        break

In [ ]:
# FOR APK-DL.COM

qTrial = Queue()
qTrial.put("cheating+wife")
qTrial.put("hidden+tracker+on+my+husband+cell+phone")
qTrial.put("phone+controller+app")
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'q': word};
    payload_str = "&".join("%s=%s" % (k,v) for k,v in payload.items())
    r = requests.get('https://apk-dl.com/search', params=payload_str);
    soup = BeautifulSoup(r.text, 'html.parser')
#     print(soup.prettify())
    names_table = soup.find_all("div", attrs={"class": "card no-rationale square-cover apps small"})
    appIDList = ""
    first = 0
    for name in names_table:
        appIDPart = name.find("a", attrs={"class": "card-click-target"})
        appID = appIDPart['href']
#         print(appID)
        imageLinkPart = name.find("img", attrs={"class": "cover-image lazy"})
        imageLink = imageLinkPart['data-original']
#         print(imageLink)
        titlePart = name.find("a", attrs={"class": "title"})
        title = titlePart.get_text()
#         print(title)
        developerNamePart = name.find("a", attrs={"class": "subtitle"})
        developerName = developerNamePart.get_text()
#         print(developerName)
        starsPart = name.find("div", attrs={"class" : "current-rating"})
        stars = starsPart['style']
        stars = stars.rsplit(' ', 1)[1]
        stars = stars[:-2]
        stars = int(stars)/10
#         print(stars)
        
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1

        appTableEntry = (appID, title, 'null', stars, imageLink, developerName, 'apk-dl.com')
        insertIntoAppDetails(conn, appTableEntry)
    
    appMainTableEntry = (word, appIDList, 'null', 'apk-dl.com')
    insertIntoAppDetailsMainTable(conn, appMainTableEntry)
    numberOfTerms = numberOfTerms + 1
    if(numberOfTerms == 5000):
        break
    

Starting couple+tracker+-+sms+appel+gps 0 with queue length 1556
Starting keep+track+of+my+kids+text+messages 1 with queue length 1555
Starting sync+facebook+contacts+to+phone 2 with queue length 1554
Starting automatic+call+record+app+lock 3 with queue length 1553
Starting ps4+remote+controller+app+to+control+phone 4 with queue length 1552
Starting track+text+messages+from+another+phone+secretly 5 with queue length 1551
Starting family+tracker+and+monitor+free 6 with queue length 1550
Starting track+husband's+phone 7 with queue length 1549
Starting free+call+and+sms+tracker 8 with queue length 1548
Starting gps+vehicle+tracking+device 9 with queue length 1547
Starting t+mobile+family+tracker 10 with queue length 1546
Starting free+calls+and+sms+tracker+app 11 with queue length 1545
Starting track+mobile+phone+exact+location+and+read+sms+and+view+call+log 12 with queue length 1544
Starting call+recording+hide+application 13 with queue length 1543
Starting police+gps+tracking+app 14 wit

Starting track+a+phone+number+by+gps 120 with queue length 1436
Starting find+my+phone+for+apple 121 with queue length 1435
Starting spy+gps+my+wife+phone+tracking 122 with queue length 1434
Starting child+tracker+apps 123 with queue length 1433
Starting call+recording+application+call+recording+application 124 with queue length 1432
Starting Call+recorder 125 with queue length 1431
Starting Call+recorder+automatic 126 with queue length 1430
Starting Auto+Call+Recorder+Pro+2017-18 127 with queue length 1429
Starting voice+call+recording+app+free+download 128 with queue length 1428
Starting track+mobile+live+location+by+number 129 with queue length 1427
Starting call+recording+app+for+samsung+j7max 130 with queue length 1426
Starting find+location+by+cell+phone+number 131 with queue length 1425
Starting family+gps+tracker+and+chat+++baby+monitor+online 132 with queue length 1424
Starting control+my+kids+phone 133 with queue length 1423
Starting call+recorder+automatically+2018 134 with 

Starting find+phone+location+by+imei+number 241 with queue length 1315
Starting call+recorder+automatic+free 242 with queue length 1314
Starting Find+My+Phone+by+Whistle 243 with queue length 1313
Starting cheater+app+spy+on+another+phone+free 244 with queue length 1312
Starting find+my+lost+phone+app 245 with queue length 1311
Starting sms+tracker+app 246 with queue length 1310
Starting backuptrans+sms+sync 247 with queue length 1309
Starting call+recorder+pro+version 248 with queue length 1308
Starting track+cell+phone+usage 249 with queue length 1307
Starting track+your+boyfriends+cell+phone 250 with queue length 1306
Starting track+mobile+location+by+number+gps 251 with queue length 1305
Starting i+want+track+mobile+number+location+apps 252 with queue length 1304
Starting Find+My+Phone+4+Android+Wear 253 with queue length 1303
Starting cheating+husband+gps+tracker+app 254 with queue length 1302
Starting call+record+app+downloading 255 with queue length 1301
Starting find+my+phone+a

Starting find+my+phone+from+google 363 with queue length 1193
Starting free+family+tracker+and+locator 364 with queue length 1192
Starting free+cheaters+app 365 with queue length 1191
Starting mysms+sms+text+messaging+sync+tablet 366 with queue length 1190
Starting gps+tracking+devices 367 with queue length 1189
Starting automatic+call+recorder+new+2017 368 with queue length 1188
Starting mobile+spy+keylogger 369 with queue length 1187
Starting how+to+catch+a+cheating+spouse 370 with queue length 1186
Starting app+to+track+phone+usage 371 with queue length 1185
Starting call+recording+app+free 372 with queue length 1184
Starting cell+phone+tracker+track+a+cell+phone+by+number+free+other+people's+phone+without+them+knowing+spy 373 with queue length 1183
Starting Pro+Call+Recorder+2018 374 with queue length 1182
Starting track+gps+by+phone+number 375 with queue length 1181
Starting call+recorder 376 with queue length 1180
Starting all+voice+call+recorder+automatically 377 with queue leng

Starting call+recorder+for+samsung+s9+plus 484 with queue length 1072
Starting call+record+app+free+download 485 with queue length 1071
Starting smart+call+recorder+pro+version 486 with queue length 1070
Starting find+me+a+free+movie+app+where+i+can+watch+movies+and+tv+shows+on+my+phone+without+paying 487 with queue length 1069
Starting free+apps+to+catch+cheaters 488 with queue length 1068
Starting track+a+cell+phone+free+app 489 with queue length 1067
Starting call+record+application+download 490 with queue length 1066
Starting track+my+husbands+phone+without+him+knowing 491 with queue length 1065
Starting call+recorder+for+samsung+galaxy+s8+plus 492 with queue length 1064
Starting att+message+backup+and+sync 493 with queue length 1063
Starting track+my+kids+phones+with+gps+without+them+knowing 494 with queue length 1062
Starting track+my+friends+phone+by+number 495 with queue length 1061
Starting auto+call+recording+application 496 with queue length 1060
Starting Call+Recorder+-+Aut

...

Starting call+recorder+for+samsung+s8+plus 969 with queue length 587
Starting phone+number+live+tracker+with+location 970 with queue length 586
Starting listen+to+people's+phone+call+spy+app 971 with queue length 585
Starting phone+sync+nextcloud 972 with queue length 584
Starting call+and+sms+tracke 973 with queue length 583
Starting is+my+wife+cheating+on+me 974 with queue length 582
Starting call+recorder+for+samsung+a7+2017 975 with queue length 581
Starting androrec+free+automatic+call+recorder 976 with queue length 580
Starting mobile+location+tracker+application 977 with queue length 579
Starting gps+cell+phone+tracker+track+a+cell+phone+by+number+free+without+installing+an+app+on+persons+phone 978 with queue length 578
Starting i+want+track+mobile+number+location+app 979 with queue length 577
Starting control+tv+with+phone+lg 980 with queue length 576
Starting track+any+cell+phone+location+by+number 981 with queue length 575
Starting call+recorder+-+automaticcall+recording 982 

Starting call+recorder+\u043e\u0442+appliqato 1084 with queue length 472
Starting auto+call+recorder+app+downloading 1085 with queue length 471
Starting call+recorder+app+for+samsung+j2 1086 with queue length 470
Starting track+my+phone+by+number 1087 with queue length 469
Starting track+my+phone+by+phone+number 1088 with queue length 468
Starting auto+call+record+app+download 1089 with queue length 467
Starting automatic+call+recorder+for+samsung+s8 1090 with queue length 466
Starting track+spouse's+phone 1091 with queue length 465
Starting appliqato+automatic+call+recorder 1092 with queue length 464
Starting apps+to+catch+cheating+husbands 1093 with queue length 463
Starting call+recorder+-+automatic+call+recorder 1094 with queue length 462
Starting app+to+catch+cheaters 1095 with queue length 461
Starting spy+on+my+wife 1096 with queue length 460
Starting how+to+catch+a+cheating+boyfriend 1097 with queue length 459
Starting SMS+from+Tablet+&+MMS+Text+Messaging+Sync 1098 with queue l

Starting imo+call+recorder+automatic+2018 1205 with queue length 351
Starting find+my+phone+by+serial+number 1206 with queue length 350
Starting track+husband 1207 with queue length 349
Starting family+gps+tracker+kid+control 1208 with queue length 348
Starting track+mobile+location+by+emi+number 1209 with queue length 347
Starting Catch+cheating+lover 1210 with queue length 346
Starting call+record+apps+lock 1211 with queue length 345
Starting free+app+to+spy+on+your+other+text+messages+to+see+if+my+girlfriend+i+phone+call+log 1212 with queue length 344
Starting voice+call+record+app+download 1213 with queue length 343
Starting trace+\u2013+email+tracking 1214 with queue length 342
Starting Track+-+Email+Tracking 1215 with queue length 341
Starting find+my+android+phone 1216 with queue length 340
Starting track+a+phone+by+number+app 1217 with queue length 339
Starting app+to+track+location+of+another+mobile 1218 with queue length 338
Starting track+my+phone+app+by+email 1219 with queu

Starting find+my+phone+android+app 1327 with queue length 229
Starting Automatic+Call+Recorder+(ACR)-+Call+Recorder+Free 1328 with queue length 228
Starting phone+location+track+apps 1329 with queue length 227
Starting smart+controller+iphone+inoty 1330 with queue length 226
Starting best+call+recorder+for+android+2018 1331 with queue length 225
Starting tracker+app+for+family 1332 with queue length 224
Starting how+to+control+your+tv+with+your+phone 1333 with queue length 223
Starting track+mobile+location+by+number+without+calling 1334 with queue length 222
Starting spy+my+boyfriend 1335 with queue length 221
Starting automatic+call+recording+2ways+-+call+recorder+acr 1336 with queue length 220
Starting mmguardian+parental+control+app+for+kids+phone 1337 with queue length 219
Starting call+recorder+hide+app+cloud+backup+clear+sound 1338 with queue length 218
Starting listen+to+cell+phone+calls 1339 with queue length 217
Starting automatic+call+record+download+app 1340 with queue leng

Starting phone+location+track+app 1447 with queue length 109
Starting Video+Call+Recorder+-+Automatic+Call+Recorder+Free 1448 with queue length 108
Starting call+recorder+automatic 1449 with queue length 107
Starting free+calls+and+sms+track 1450 with queue length 106
Starting track+location+by+imei+number 1451 with queue length 105
Starting free+keylogger+app 1452 with queue length 104
Starting call+recorder+for+android+2017 1453 with queue length 103
Starting track+mobile+location+by+number+live 1454 with queue length 102
Starting call+record+apps+download 1455 with queue length 101
Starting track+a+phone+by+sim+number 1456 with queue length 100
Starting free+keylogger+app+for+android 1457 with queue length 99
Starting call+recorder+for+android 1458 with queue length 98
Starting track+phone+number+app 1459 with queue length 97
Starting track+my+children 1460 with queue length 96
Starting cheating+boyfriend+game 1461 with queue length 95
Starting listen+to+call 1462 with queue length 